# Custom Prediction Routines for Endpoints That Train

- https://cloud.google.com/vertex-ai/docs/predictions/custom-prediction-routines
- https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform.prediction.LocalModel
- https://github.com/googleapis/python-aiplatform/blob/custom-prediction-routine/google/cloud/aiplatform/prediction/predictor.py
- https://github.com/googleapis/python-aiplatform/blob/custom-prediction-routine/google/cloud/aiplatform/prediction/sklearn/predictor.py
- https://codelabs.developers.google.com/vertex-cpr-sklearn#0

In [1]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

'statmike-mlops-349915'

In [2]:
REGION = 'us-central1'
EXPERIMENT = 'cpr_training'
SERIES = '04'

# source data
BQ_PROJECT = PROJECT_ID
BQ_DATASET = 'fraud'
BQ_TABLE = 'fraud_prepped'

# Resources
DEPLOY_COMPUTE = 'n1-standard-2'

# Model Training
VAR_TARGET = 'Class'
VAR_OMIT = 'transaction_id' # add more variables to the string with space delimiters

In [3]:
from google.cloud import aiplatform
from google.cloud import bigquery
from google.cloud import service_usage_v1

from datetime import datetime
import json
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import gridspec
from sklearn import metrics

from google.protobuf import json_format
from google.protobuf.struct_pb2 import Value

In [4]:
aiplatform.init(project=PROJECT_ID, location=REGION)
bq = bigquery.Client(project = PROJECT_ID)

In [5]:
BUCKET = PROJECT_ID
URI = f"gs://{BUCKET}/{SERIES}/{EXPERIMENT}"
REPOSITORY = f"{REGION}-docker.pkg.dev/{PROJECT_ID}/{PROJECT_ID}"
DIR = f"temp/{EXPERIMENT}"

In [6]:
SERVICE_ACCOUNT = !gcloud config list --format='value(core.account)' 
SERVICE_ACCOUNT = SERVICE_ACCOUNT[0]
SERVICE_ACCOUNT

'1026793852137-compute@developer.gserviceaccount.com'

In [7]:
!rm -rf {DIR}
!mkdir -p {DIR}

## Idea: Decision Tree on Samples

- Input parameter is a sample size `n`
- Retrieve a sample of size `n` from a BigQuery table to a Pandas dataframe
- Use sklearn.tree.DecisionTreeClassifier to build a classifier
- Retrieve the rules of the tree

In [431]:
n = 4000

In [432]:
train = bq.query(query = f"""
        SELECT * EXCEPT(splits, transaction_id)
            FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}`
            WHERE splits = 'TRAIN' and RAND() < 0.1
            LIMIT {n}
        """).to_dataframe()
y = train[VAR_TARGET]
X = train.drop(VAR_TARGET, axis = 1)

In [433]:
from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier(max_leaf_nodes = 3, random_state = 0)
clf.fit(X, y)

DecisionTreeClassifier(max_leaf_nodes=3, random_state=0)

In [434]:
n_nodes = clf.tree_.node_count
children_left = clf.tree_.children_left
children_right = clf.tree_.children_right
feature = clf.tree_.feature
threshold = clf.tree_.threshold

def path_builder(node_num, path, x):
        path.append(node_num)
        if node_num == x:
            return True
        left = False
        right = False
        if (children_left[node_num] !=-1):
            left = path_builder(children_left[node_num], path, x)
        if (children_right[node_num] !=-1):
            right = path_builder(children_right[node_num], path, x)
        if left or right :
            return True
        path.remove(node_num)
        return False


def rule_builder(path, column_names):
    rule = ''
    for index, node in enumerate(path):
        if index != len(path)-1:
            if len(rule) > 0: rule += ' and '
            if (children_left[node] == path[index+1]):
                rule += f"{column_names[feature[node]]} <= {threshold[node]}"
            else:
                rule += f"{column_names[feature[node]]} > {threshold[node]}"
    return rule

paths ={}
for leaf in np.unique(clf.apply(X)):
    path_leaf = []
    path_builder(0, path_leaf, leaf)
    paths[leaf] = np.unique(np.sort(path_leaf))

rules = {}
for key in paths:
    rules[key] = rule_builder(paths[key], X.columns)

rules

{2: 'V14 > -4.16540265083313',
 3: 'V14 <= -4.16540265083313 and V6 <= -0.5753848850727081',
 4: 'V14 <= -4.16540265083313 and V6 > -0.5753848850727081'}

---
## Build Custom Prediction Routine

A custom container built by the Vertex AI SDK that assist with pre/post processing code without the need to setup an HTTP server.

In [32]:
!pip install google-cloud-aiplatform[prediction] -U -q

In [436]:
!mkdir -p {DIR}/SRC

In [437]:
%%writefile {DIR}/SRC/requirements.txt
fastapi
uvicorn==0.17.6
#joblib~=1.0
numpy~=1.20
scikit-learn~=0.24
pandas
#google-cloud-storage>=1.26.0,<2.0.0dev
google-cloud-aiplatform[prediction]>=1.16.0
google-cloud-bigquery
pyarrow

Writing temp/cpr_training/SRC/requirements.txt


In [441]:
%%writefile {DIR}/SRC/predictor.py

# packages
import numpy as np
import json
from sklearn.tree import DecisionTreeClassifier
from google.cloud.aiplatform.prediction.predictor import Predictor
from google.cloud import bigquery

##################################################################################################

# clients
bq = bigquery.Client(project = 'statmike-mlops-349915')

# source data
BQ_PROJECT = 'statmike-mlops-349915'
BQ_DATASET = 'fraud'
BQ_TABLE = 'fraud_prepped'

# Model Training
VAR_TARGET = 'Class'

def ruler(n):
    # helper function: 
    def path_builder(node_num, path, x):
            path.append(node_num)
            if node_num == x:
                return True
            left = False
            right = False
            if (children_left[node_num] !=-1):
                left = path_builder(children_left[node_num], path, x)
            if (children_right[node_num] !=-1):
                right = path_builder(children_right[node_num], path, x)
            if left or right :
                return True
            path.remove(node_num)
            return False
    # helper function:
    def rule_builder(path, column_names):
        rule = ''
        for index, node in enumerate(path):
            if index != len(path)-1:
                if len(rule) > 0: rule += ' and '
                if (children_left[node] == path[index+1]):
                    rule += f"{column_names[feature[node]]} <= {threshold[node]}"
                else:
                    rule += f"{column_names[feature[node]]} > {threshold[node]}"
        return rule
    
    # data
    train = bq.query(query = f"""
        SELECT * EXCEPT(splits, transaction_id)
            FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}`
            WHERE splits = 'TRAIN' and RAND() < 0.1
            LIMIT {n}
        """).to_dataframe()
    y = train[VAR_TARGET]
    X = train.drop(VAR_TARGET, axis = 1)
    
    # model
    clf = DecisionTreeClassifier(max_leaf_nodes = 3, random_state = 0)
    clf.fit(X, y)
    
    # outputs
    n_nodes = clf.tree_.node_count
    children_left = clf.tree_.children_left
    children_right = clf.tree_.children_right
    feature = clf.tree_.feature
    threshold = clf.tree_.threshold
    
    # decision
    paths ={}
    for leaf in np.unique(clf.apply(X)):
        path_leaf = []
        path_builder(0, path_leaf, leaf)
        paths[leaf] = np.unique(np.sort(path_leaf))
    rules = {}
    for key in paths:
        rules[key] = rule_builder(paths[key], X.columns)

    return rules

##################################################################################################

class CprPredictor(Predictor):
    def __init__(self):
        return

    def load(self, artifacts_uri: str) -> None:
        # no model to load here, this example trains a model and returns parameters
        pass

    def predict(self, instances):

        instances = instances["instances"]
        results = [f"{ruler(instance)}" for instance in instances]
        
        return {"predictions": results}

Overwriting temp/cpr_training/SRC/predictor.py


In [442]:
from google.cloud.aiplatform.prediction import LocalModel
# load the local predictor class:
from temp.cpr_training.SRC.predictor import CprPredictor

local_model = LocalModel.build_cpr_model(
    src_dir = f"{DIR}/SRC",
    output_image_uri = f"{REPOSITORY}/{SERIES}_{EXPERIMENT}",
    predictor = CprPredictor,
    requirements_path = os.path.join(f"{DIR}/SRC", "requirements.txt"),
)

In [443]:
with local_model.deploy_to_local_endpoint() as local_endpoint:
    predict_response = local_endpoint.predict(
        request = '{"instances": [100, 1000, 2000, 3000]}',
        headers={"Content-Type": "application/json"}
    )

    health_check_response = local_endpoint.run_health_check()

In [444]:
[print(r+'\n') for r in json.loads(predict_response.content)['predictions']]

{1: 'V17 <= -2.0364582538604736', 2: 'V17 > -2.0364582538604736'}

{2: 'V11 > 3.273556113243103', 3: 'V11 <= 3.273556113243103 and V11 <= 2.1354647874832153', 4: 'V11 <= 3.273556113243103 and V11 > 2.1354647874832153'}

{1: 'V11 <= 3.220276713371277', 2: 'V11 > 3.220276713371277'}

{1: 'V11 <= 3.5032249689102173', 2: 'V11 > 3.5032249689102173'}



[None, None, None, None]

In [445]:
!gcloud auth configure-docker {REGION}-docker.pkg.dev --quiet


{
  "credHelpers": {
    "gcr.io": "gcloud",
    "us.gcr.io": "gcloud",
    "eu.gcr.io": "gcloud",
    "asia.gcr.io": "gcloud",
    "staging-k8s.gcr.io": "gcloud",
    "marketplace.gcr.io": "gcloud",
    "us-central1-docker.pkg.dev": "gcloud"
  }
}
Adding credentials for: us-central1-docker.pkg.dev
gcloud credential helpers already registered correctly.


In [446]:
local_model.push_image()

In [447]:
model = aiplatform.Model.upload(
    local_model = local_model,
    display_name = f"{SERIES}_{EXPERIMENT}"
)

Creating Model
Create Model backing LRO: projects/1026793852137/locations/us-central1/models/6342900061709533184/operations/8112824954468171776
Model created. Resource name: projects/1026793852137/locations/us-central1/models/6342900061709533184@1
To use this Model in another session:
model = aiplatform.Model('projects/1026793852137/locations/us-central1/models/6342900061709533184@1')


In [70]:
model = aiplatform.Model('projects/1026793852137/locations/us-central1/models/6342900061709533184@1')

In [448]:
endpoint = model.deploy(
    machine_type = DEPLOY_COMPUTE,
    min_replica_count = 1,
    max_replica_count = 5,
    service_account = SERVICE_ACCOUNT
)

Creating Endpoint
Create Endpoint backing LRO: projects/1026793852137/locations/us-central1/endpoints/4685331307255824384/operations/5894802137988202496
Endpoint created. Resource name: projects/1026793852137/locations/us-central1/endpoints/4685331307255824384
To use this Endpoint in another session:
endpoint = aiplatform.Endpoint('projects/1026793852137/locations/us-central1/endpoints/4685331307255824384')
Deploying model to Endpoint : projects/1026793852137/locations/us-central1/endpoints/4685331307255824384
Deploy Endpoint model backing LRO: projects/1026793852137/locations/us-central1/endpoints/4685331307255824384/operations/845141055799033856
Endpoint model deployed. Resource name: projects/1026793852137/locations/us-central1/endpoints/4685331307255824384


In [8]:
endpoint = aiplatform.Endpoint('projects/1026793852137/locations/us-central1/endpoints/4685331307255824384')

In [9]:
predictions = endpoint.predict(instances = [100, 1000, 2000])
predictions

Prediction(predictions=["{1: 'V7 <= -6.2548463344573975', 2: 'V7 > -6.2548463344573975'}", "{1: 'V10 <= -3.2568970918655396', 3: 'V10 > -3.2568970918655396 and V19 <= -1.4964446425437927', 4: 'V10 > -3.2568970918655396 and V19 > -1.4964446425437927'}", "{1: 'V3 <= -5.613183259963989', 2: 'V3 > -5.613183259963989'}"], deployed_model_id='7424619392324861952', model_version_id='1', model_resource_name='projects/1026793852137/locations/us-central1/models/6342900061709533184', explanations=None)

In [10]:
len(predictions.predictions)

3

In [11]:
endpoint.resource_name

'projects/1026793852137/locations/us-central1/endpoints/4685331307255824384'

In [12]:
# get the Async Client for the endpoint:
from google.cloud import aiplatform_v1

client_options = {"api_endpoint": f"{REGION}-aiplatform.googleapis.com"}
parent = f"projects/{PROJECT_ID}/locations/{REGION}"

client = aiplatform_v1.PredictionServiceAsyncClient(client_options = client_options)

In [13]:
instance = [1000]

In [14]:
import asyncio
import time

In [15]:
predictions = await client.predict(endpoint = endpoint.resource_name, instances = instance)
predictions

predictions {
  string_value: "{1: \'V3 <= -5.189486265182495\', 2: \'V3 > -5.189486265182495\'}"
}
deployed_model_id: "7424619392324861952"
model: "projects/1026793852137/locations/us-central1/models/6342900061709533184"
model_display_name: "04_cpr_training"
model_version_id: "1"

In [16]:
predictions.predictions

["{1: 'V3 <= -5.189486265182495', 2: 'V3 > -5.189486265182495'}"]

In [17]:
len(predictions.predictions)

1

In [18]:
from google.api_core.retry import Retry
from google.api_core import exceptions

_RETRIABLE_TYPES = [
    exceptions.InternalServerError,
    exceptions.BadGateway,
    exceptions.TooManyRequests,
    exceptions.ServiceUnavailable,
]

def is_retryable(exc):
    return isinstance(exc, _RETRIABLE_TYPES)

my_retry_policy = Retry(predicate = is_retryable)

In [531]:
async def asyncPredictions(batch_size = 1, concur_requests = 10, total_instances = 100):
    limit = asyncio.Semaphore(concur_requests)
    instance = [1000]
    predictions = [None] * total_instances

    async def predictor(p, batch_size):
        async with limit:
            if limit.locked():
                await asyncio.sleep(.01)
            prediction = await client.predict(
                endpoint = endpoint.resource_name, 
                instances = instance * batch_size,
                retry = my_retry_policy,
                timeout = 100000
            )

        predictions[p:p+batch_size] = prediction.predictions

    async def runner(batch_size, total_instances):
        tasks = []
        for p in range(0, total_instances, batch_size):
            task = asyncio.create_task(predictor(p, batch_size))
            tasks.append(task)
        results = await asyncio.gather(*tasks)

    start = time.perf_counter()
    await runner(batch_size, total_instances)
    elapsed = time.perf_counter() - start
    print(f'{elapsed:0.5f} seconds')
    
    return predictions

In [522]:
predictions = await asyncPredictions(1, 2, 10)

5.73065 seconds


In [483]:
len(predictions)

10

In [484]:
predictions

["{1: 'V11 <= 2.903998017311096', 2: 'V11 > 2.903998017311096'}",
 "{1: 'V10 <= -2.856087803840637', 3: 'V10 > -2.856087803840637 and V14 <= -1.8680251240730286', 4: 'V10 > -2.856087803840637 and V14 > -1.8680251240730286'}",
 "{1: 'V17 <= -3.4257320165634155', 2: 'V17 > -3.4257320165634155'}",
 "{1: 'V11 <= 3.1742337942123413', 2: 'V11 > 3.1742337942123413'}",
 "{1: 'V14 <= -5.684613943099976', 2: 'V14 > -5.684613943099976'}",
 "{1: 'V14 <= -4.659088969230652', 2: 'V14 > -4.659088969230652'}",
 "{1: 'V17 <= -4.50637024641037', 2: 'V17 > -4.50637024641037'}",
 "{1: 'V4 <= 5.306996822357178', 2: 'V4 > 5.306996822357178'}",
 "{1: 'V3 <= -5.582198619842529', 2: 'V3 > -5.582198619842529'}",
 "{1: 'V10 <= -2.866185188293457', 2: 'V10 > -2.866185188293457'}"]

In [485]:
predictions = await asyncPredictions(1, 4, 20)

10.38664 seconds


In [486]:
len(predictions)

20

In [489]:
predictions = await asyncPredictions(10, 4, 40)

29.90747 seconds


In [490]:
len(predictions)

40

In [525]:
predictions = await asyncPredictions(10, 4, 100)

65.90143 seconds


In [526]:
len(predictions)

100

In [527]:
predictions = await asyncPredictions(10, 5, 100)

62.64738 seconds


In [528]:
len(predictions)

100

In [495]:
predictions = await asyncPredictions(10, 6, 100)

50.71739 seconds


In [496]:
len(predictions)

100

In [497]:
predictions = await asyncPredictions(10, 7, 100)

60.53520 seconds


In [498]:
len(predictions)

100

In [506]:
predictions = await asyncPredictions(10, 8, 100)

60.52704 seconds


In [507]:
len(predictions)

100

In [49]:
async def predictor(batch_size):
    prediction = await client.predict(
        endpoint = endpoint.resource_name, 
        instances = [1000] * batch_size,
        retry = my_retry_policy,
        timeout = 100000)
    return prediction.predictions[0]

async def runner(p):
    tasks = [predictor(1) for _ in range(p)]
    return await asyncio.gather(*tasks)

In [50]:
results = await runner(50)

In [51]:
len(results)

50

In [52]:
results

["{1: 'V11 <= 3.928184151649475', 2: 'V11 > 3.928184151649475'}",
 "{1: 'V7 <= -10.85008955001831', 2: 'V7 > -10.85008955001831'}",
 "{1: 'V17 <= -6.344502687454224', 3: 'V17 > -6.344502687454224 and V7 <= 1.281801998615265', 4: 'V17 > -6.344502687454224 and V7 > 1.281801998615265'}",
 "{1: 'V11 <= 2.1383026838302612', 3: 'V11 > 2.1383026838302612 and V13 <= -1.161233365535736', 4: 'V11 > 2.1383026838302612 and V13 > -1.161233365535736'}",
 "{1: 'V14 <= -4.2042152881622314', 3: 'V14 > -4.2042152881622314 and V13 <= -2.43116557598114', 4: 'V14 > -4.2042152881622314 and V13 > -2.43116557598114'}",
 "{2: 'V17 > 5.3876073360443115', 3: 'V17 <= 5.3876073360443115 and V11 <= 2.1229100227355957', 4: 'V17 <= 5.3876073360443115 and V11 > 2.1229100227355957'}",
 "{1: 'V17 <= -7.099818468093872', 2: 'V17 > -7.099818468093872'}",
 "{1: 'V10 <= -2.821014881134033', 3: 'V10 > -2.821014881134033 and V11 <= 1.909305453300476', 4: 'V10 > -2.821014881134033 and V11 > 1.909305453300476'}",
 "{1: 'V11 <= 

In [53]:
results = []
for l in range(5):
    results += await runner(10)

In [54]:
len(results)

50

In [55]:
results

["{1: 'V17 <= -4.027241945266724', 2: 'V17 > -4.027241945266724'}",
 "{0: ''}",
 "{1: 'V14 <= -4.285696983337402', 3: 'V14 > -4.285696983337402 and V7 <= 1.2747080326080322', 4: 'V14 > -4.285696983337402 and V7 > 1.2747080326080322'}",
 "{1: 'V17 <= -6.415940046310425', 3: 'V17 > -6.415940046310425 and V7 <= 1.2888912558555603', 4: 'V17 > -6.415940046310425 and V7 > 1.2888912558555603'}",
 "{1: 'V3 <= -4.849050283432007', 2: 'V3 > -4.849050283432007'}",
 "{1: 'V17 <= -3.5062111020088196', 2: 'V17 > -3.5062111020088196'}",
 "{1: 'V11 <= 3.0515365600585938', 2: 'V11 > 3.0515365600585938'}",
 "{1: 'V3 <= -5.447434186935425', 2: 'V3 > -5.447434186935425'}",
 "{1: 'V10 <= -2.6969146728515625', 3: 'V10 > -2.6969146728515625 and V14 <= -1.6223512887954712', 4: 'V10 > -2.6969146728515625 and V14 > -1.6223512887954712'}",
 "{1: 'V17 <= -2.704252779483795', 3: 'V17 > -2.704252779483795 and V11 <= 1.9256085753440857', 4: 'V17 > -2.704252779483795 and V11 > 1.9256085753440857'}",
 "{1: 'V17 <= -4.

In [56]:
results = []
for l in range(5):
    results += await runner(20)

In [57]:
len(results)

100

In [58]:
results = []
for l in range(2):
    results += await runner(50)

In [59]:
len(results)

100

In [62]:
async def predictor(batch_size, n):
    prediction = await client.predict(
        endpoint = endpoint.resource_name, 
        instances = [1000] * batch_size,
        retry = my_retry_policy,
        timeout = 100000)
    return prediction.predictions[0]

async def runner(p, batch_size, n):
    tasks = [predictor(batch_size, n) for _ in range(p)]
    return await asyncio.gather(*tasks)

In [63]:
results = []
for l in range(2):
    results += await runner(50, 1, 10000)

In [64]:
len(results)

100

In [66]:
results = []
for l in range(2):
    results += await runner(75, 1, 20000)

ServiceUnavailable: 503 502:Bad Gateway

In [ ]:
len(results)

In [67]:
endpoint.delete(force = True)

Undeploying Endpoint model: projects/1026793852137/locations/us-central1/endpoints/4685331307255824384
Undeploy Endpoint model backing LRO: projects/1026793852137/locations/us-central1/endpoints/4685331307255824384/operations/5021103810278326272
Endpoint model undeployed. Resource name: projects/1026793852137/locations/us-central1/endpoints/4685331307255824384
Deleting Endpoint : projects/1026793852137/locations/us-central1/endpoints/4685331307255824384
Delete Endpoint  backing LRO: projects/1026793852137/locations/us-central1/operations/22108223897075712
Endpoint deleted. . Resource name: projects/1026793852137/locations/us-central1/endpoints/4685331307255824384


In [71]:
model.delete()

Deleting Model : projects/1026793852137/locations/us-central1/models/6342900061709533184
Delete Model  backing LRO: projects/1026793852137/locations/us-central1/operations/5901276062452547584
Model deleted. . Resource name: projects/1026793852137/locations/us-central1/models/6342900061709533184
